# Generating a weighted Monte Carlo lightcone of Diffsky galaxies

This notebook demonstrates how to generate a lightcone of diffsky galaxies with SEDs, star formation histories, dust, and other properties.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

### Download stellar population synthesis data

The very first thing we will do in this notebook is download some LSST-like transmission curves, and also some SEDs of simple stellar populations (SSPs) used in the SED modeling. You can skip this step of you have already cached previously-stored SSP data as described in the [DSPS Quickstart Guide](https://dsps.readthedocs.io/en/latest/quickstart.html).

In [ ]:
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/ssp_data_fsps_v3.2_lgmet_age.h5

In [ ]:
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/filters/lsst_u_transmission.h5
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/filters/lsst_g_transmission.h5
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/filters/lsst_r_transmission.h5
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/filters/lsst_i_transmission.h5
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/filters/lsst_z_transmission.h5
! wget https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/filters/lsst_y_transmission.h5

#### The `load_ssp_templates` function in DSPS loads an hdf5 file and packs it into a namedtuple with the expected field names.

In [ ]:
from dsps.data_loaders import load_ssp_templates
ssp_data = load_ssp_templates(fn="ssp_data_fsps_v3.2_lgmet_age.h5")

print(ssp_data._fields)

#### The `load_transmission_curve` function in DSPS loads an hdf5 file and packs it into a namedtuple with the expected field names.

In [ ]:
from dsps.data_loaders import load_transmission_curve
fn_pat = "lsst_{}_transmission.h5"
filter_names = ('u', 'g', 'r', 'i', 'z', 'y')
fn_list = [fn_pat.format(x) for x in filter_names]
tcurves = [load_transmission_curve(fn) for fn in fn_list]

print(tcurves[0]._fields)

#### Generate the halo lightcone and additional data needed to model/compute SEDs

First specify halo lightcone specs

In [ ]:
from diffsky.experimental import mc_weighted_lightcone_data

num_halos = 100
z_min, z_max = 0.2, 3.0
lgmp_min, lgmp_max = 10.5, 15.0
sky_area_degsq = 1.0

#### Get a random number seed from JAX

In [ ]:
from jax import random as jran
ran_key = jran.key(0)

#### Define a redshift table used for photometry interpolation

In [ ]:
n_z_phot_table = 30
z_phot_table = np.linspace(z_min, z_max, n_z_phot_table)

#### Generate the lightcone of dark matter halos and additional SPS data

In [ ]:
halo_lc_data = (num_halos, z_min, z_max, lgmp_min, lgmp_max, sky_area_degsq)
phot_data = (ssp_data, tcurves, z_phot_table)

ran_key, lc_halo_key = jran.split(ran_key, 2)
args = (lc_halo_key, *halo_lc_data, *phot_data)

lc_data = mc_weighted_lightcone_data(*args)

#### Populate the lightcone with diffsky galaxies

In [ ]:
from diffsky.experimental import mc_weighted_diffsky_lightcone

ran_key, sed_key = jran.split(ran_key, 2)
sed_info = mc_weighted_diffsky_lightcone(sed_key, lc_data)
sed_info.keys()

#### For the _weighted_ lightcone, each halo has multiplicity according to its abundance in the volume

This `nhalos` column needs to be taken into account when predicting summary statistics from the weighted lightcone.

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.loglog()
__=ax.scatter(10**sed_info['logsm_obs'], lc_data.nhalos, s=1)
xlabel = ax.set_xlabel(r'$M_{\star}\ [M_{\odot}]$')
ylabel = ax.set_ylabel(r'$N_{\rm halos}$')

#### Calculate the halo mass function, accounting for halo weights

The unweighted version uniformly spans $\log_{10}M_{\rm halo}$. The weighted version has the expected Schechter-type shape of the HMF.

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
__=ax.hist(sed_info['logmp_obs'], bins=100, alpha=0.7, label=r'${\rm unweighted}$')
__=ax.hist(sed_info['logmp_obs'], bins=100, weights=lc_data.nhalos, alpha=0.7, label=r'${\rm weighted}$')
xlabel = ax.set_xlabel(r'$\log_{10}M_{\rm halo}/M_{\odot}$')
ylabel = ax.set_ylabel(r'$N_{\rm halos}$')
leg = ax.legend()

#### Visually inspect the diversity of SFHs

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
ylim = ax.set_ylim(8e-3, 5e2)
xscale = ax.set_xscale('log')
xlim = ax.set_xlim(1, 15)

n_plot = 5
for i in range(n_plot):
    __=ax.plot(lc_data.t_table, sed_info['sfh_table'][i, :])

xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'${\rm SFR\ [M_{\odot}/yr]}$')

#### Visually inspect the sSFR PDF

Note that the plot below shows the PDF for _all_ galaxies/halos in the lightcone, without accounting for the weights

In [ ]:
fig, ax = plt.subplots(1, 1)
xlim = ax.set_xlim(-14, -7)
__=ax.hist(sed_info['logssfr_obs'], bins=150, alpha=0.7)
xlabel = ax.set_xlabel(r'${\rm log_{10}(sSFR)}$')

#### Visually inspect star-forming sequence

In [ ]:
fig, ax = plt.subplots(1, 1)
ylim = ax.set_ylim(-13.5, -7.5)
xlim = ax.set_xlim(7.5, 12)
__=ax.scatter(sed_info['logsm_obs'], sed_info['logssfr_obs'], s=1)
xlabel = ax.set_xlabel(r'${\rm log_{10}(M_{\star})}$')
ylabel = ax.set_ylabel(r'${\rm log_{10}(sSFR)}$')

### Plot color-color diagram

Note that the plot below shows the PDF for _all_ galaxies/halos in the lightcone, without accounting for the weights, and without selecting a particular galaxy sample of interest.

In [ ]:
fig, ax = plt.subplots(1, 1)

ri = sed_info['obs_mags'][:, 2]-sed_info['obs_mags'][:, 3]
iz = sed_info['obs_mags'][:, 3]-sed_info['obs_mags'][:, 4]
__=ax.scatter(iz, ri, s=1)
xlabel = ax.set_xlabel(r'${\rm i-z}$')
ylabel = ax.set_ylabel(r'${\rm r-i}$')

### Plot color--redshift relation

Note that the plot below shows the PDF for _all_ galaxies/halos in the lightcone, without accounting for the weights, and without selecting a particular galaxy sample of interest.

In [ ]:
fig, ax = plt.subplots(1, 1)

iz = sed_info['obs_mags'][:, 3]-sed_info['obs_mags'][:, 4]
__=ax.scatter(lc_data.z_obs, iz, s=1)
xlabel = ax.set_xlabel(r'${\rm redshift}$')
ylabel = ax.set_ylabel(r'${\rm i-z}$')

### Examine diversity of SEDs

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
xscale = ax.set_xscale('log')

n_plot = 3
for i in range(n_plot):
    __=ax.plot(lc_data.ssp_data.ssp_wave, sed_info['rest_sed'][i, :])

xlabel = ax.set_xlabel(r'${\rm \lambda\ [\AA]}$')
ylabel = ax.set_ylabel(r'${\rm L_{\odot}/Hz}$')
title = ax.set_title(r'${\rm restframe\ SEDs}$')

xlim = ax.set_xlim(2_000, 8_000)
ymin, ymax = ax.get_ylim()
ylim = ax.set_ylim(2e-8, ymax)